# Convolutional Neural Network

### Part 1 - Building the CNN

#### Importing the Tensorflow libraries and packages

In [1]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.1.0
Keras Version: 2.2.4-tf

Python 3.7.10 (default, Feb 26 2021, 13:06:18) [MSC v.1916 64 bit (AMD64)]
Pandas 1.2.4
Scikit-Learn 0.24.1
GPU is available


In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau

import os

In [3]:
TRAIN_DIR = 'dataset/train'
TEST_DIR = 'dataset/test'
VALID_DIR = 'dataset/valid'
IMG_SIZE = 224
IMG_RESCALE = 255

In [4]:
classifier = Sequential()

#classes = 250

#Kernels usually 1x1 | 3x3 | 5x5

#Adding a convolutional layer
classifier.add(Conv2D(32, (3, 3), input_shape = (IMG_SIZE, IMG_SIZE, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#Adding a second convolutional layer
classifier.add(Conv2D(64, (3, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#Adding a third convolutional layer
classifier.add(Conv2D(128, (3, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#Adding a fourth convolutional layer
classifier.add(Conv2D(256, (3, 3), input_shape = (IMG_SIZE, IMG_SIZE, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#Flattening
classifier.add(Flatten())

#Full connection
classifier.add(Dense(units = 1024, activation = 'relu'))
classifier.add(Dropout(0.10))
classifier.add(Dense(units = 250, activation= 'softmax'))

Let's double check the model description:

In [5]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 256)       2

In [6]:
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

### Part 2 - Fitting the CNN to the images


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(rescale = 1./IMG_RESCALE,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./IMG_RESCALE)

validation_datagen = ImageDataGenerator(rescale = 1./IMG_RESCALE)

In [8]:
training_set = train_datagen.flow_from_directory(TRAIN_DIR,
                                                 target_size = (IMG_SIZE, IMG_SIZE),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(TEST_DIR,
                                            target_size = (IMG_SIZE, IMG_SIZE),
                                            batch_size = 32,
                                            class_mode = 'categorical')

validation_set = validation_datagen.flow_from_directory(VALID_DIR,
                                            target_size = (IMG_SIZE, IMG_SIZE),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 35215 images belonging to 250 classes.
Found 1250 images belonging to 250 classes.
Found 1250 images belonging to 250 classes.


In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import datetime

#rm -rf /logs/

es = EarlyStopping(monitor='val_loss', patience=80)
mc = ModelCheckpoint('best_model_6.h5', monitor='val_loss', mode='min', save_best_only=True)
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=9)

# Create a TensorBoard callback
log_dir = "logs\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [10]:
classifier.fit( training_set,
                         epochs = 150,
                         callbacks = [tensorboard_callback,es,mc,reduceLR],
                         validation_data = validation_set,
                         workers = 4)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1101 steps, validate for 40 steps
Epoch 1/150
1101/1101 [==============================] - 101s 91ms/step - loss: 4.5260 - accuracy: 0.1021 - val_loss: 3.1853 - val_accuracy: 0.2952
Epoch 2/150
1101/1101 [==============================] - 101s 92ms/step - loss: 2.9834 - accuracy: 0.3330 - val_loss: 2.1777 - val_accuracy: 0.4960
Epoch 3/150
1101/1101 [==============================] - 99s 90ms/step - loss: 2.2557 - accuracy: 0.4732 - val_loss: 1.7068 - val_accuracy: 0.5832
Epoch 4/150
1101/1101 [==============================] - 99s 90ms/step - loss: 1.8213 - accuracy: 0.5626 - val_loss: 1.4219 - val_accuracy: 0.6352
Epoch 5/150
1101/1101 [==============================] - 103s 93ms/step - loss: 1.5018 - accuracy: 0.6303 - val_loss: 1.2875 - val_accuracy: 0.6824
Epoch 6/150
1101/1101 [==============================] - 100s 91ms/step - loss: 1.2708 - accuracy: 0.6779 - val_loss: 1.2819 - val_accuracy: 0.6872
Epoch 7/150
1101/110

1101/1101 [==============================] - 106s 96ms/step - loss: 0.0652 - accuracy: 0.9803 - val_loss: 1.4240 - val_accuracy: 0.7720
Epoch 55/150
1101/1101 [==============================] - 106s 96ms/step - loss: 0.0619 - accuracy: 0.9816 - val_loss: 1.4241 - val_accuracy: 0.7720
Epoch 56/150
1101/1101 [==============================] - 105s 96ms/step - loss: 0.0611 - accuracy: 0.9822 - val_loss: 1.4241 - val_accuracy: 0.7720
Epoch 57/150
1101/1101 [==============================] - 105s 95ms/step - loss: 0.0664 - accuracy: 0.9802 - val_loss: 1.4241 - val_accuracy: 0.7720
Epoch 58/150
1101/1101 [==============================] - 109s 99ms/step - loss: 0.0660 - accuracy: 0.9802 - val_loss: 1.4241 - val_accuracy: 0.7720
Epoch 59/150
1101/1101 [==============================] - 106s 97ms/step - loss: 0.0644 - accuracy: 0.9808 - val_loss: 1.4241 - val_accuracy: 0.7720
Epoch 60/150
1101/1101 [==============================] - 107s 97ms/step - loss: 0.0679 - accuracy: 0.9800 - val_loss: 

In [11]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 6544), started 1 day, 1:03:58 ago. (Use '!kill 6544' to kill it.)

# Part 3 test the CNN

In [12]:
# Load Model
# modelo 3
classifier.load_weights('best_model_6.h5')

In [13]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 256)       2

In [14]:
score = classifier.evaluate(test_set, verbose=0)

print('Test Loss', score[0])
print('Test accuracy', score[1])

  ...
    to  
  ['...']
Test Loss 0.9688800529576838
Test accuracy 0.7536
